# Índice

01. Librerías a importar, valores constantes
02. Saturación
03. Interpolación lineal
04. Capacidad calorífica

# Librerías y valores de constantes

In [1]:
import numpy as np
import scipy 
import sympy as sp

R_gas = 8.314472    # [=] J / mol.K ; Pa.m³ / mol.K

## Saturación
- Cálculo de calidad
- Cálculo de propiedades en estado de saturación

In [ ]:
def calculo_calidad(valor_propiedad_actual : float, propiedad_estado_f : float, propiedad_estado_g : float) -> float :
    propiedad_gf = propiedad_estado_g - propiedad_estado_f
    x_calidad = (valor_propiedad_actual-propiedad_estado_f) / propiedad_gf
    return x_calidad

def valor_propiedad_saturacion(x_calidad : float, propiedad_estado_f : float, propiedad_estado_g : float):
    valor_propiedad_x = propiedad_estado_f + x_calidad * (propiedad_estado_g-propiedad_estado_f)
    return valor_propiedad_x

## Interpolación Lineal
Aplicado a estados de sustancia pura sobrecalentado / subenfriado

In [ ]:
def resultado_interpolacion(x_busqueda, x_inicial, y_inicial, x_final, y_final):
    m = (y_final-y_inicial)/(x_final-x_inicial)
    resultado = m*(x_busqueda-x_inicial) + y_inicial
    return resultado

# Capacidad calorífica 
- A presión constante (para cambios de entalpía) o volumen constante (para cambios de energía interna).
- Las Temperaturas en unidades K, y el peso molecular debe estar en g/mol.
- "c_" indica que capacidad se va a implementar, si es a presión o volumen constante.

Tomado de : "K. Wark y D. E. Richards, Termodinámica, 6.ª ed. España: McGraw-Hill Interamericana de España S.L., 2000., pp. 929."


In [ ]:
def capacidad_calorifica(
    T_i : float,
    T_f : float,
    a : float,
    b : float,
    c : float,
    d : float,
    e : float,
    PM : float | None = None,
    c_ : str = "C_p"
) -> dict:
    PM = PM * (1/1000)          # kg/mol
    T = sp.Symbol("T", real=True)
    
    if c_ not in ["C_p", "C_v"] :
        raise ValueError("c_ debe ser 'C_p' o 'C_v'")
    
    if PM <= 0:
        raise ValueError("PM debe ser > 0 (en kg/mol) para calcular J/kg")
    
    if T_i == T_f:
        if c_ == "C_p":
            return {
                "delta h [=] kJ/mol": 0.0,
                "delta h [=] kJ/kg": 0.0 if PM is not None else "PM no especificado"
            }
            
        if c_ == "C_v":
            return {
                "delta u [=] kJ/mol": 0.0,
                "delta u [=] kJ/kg": 0.0 if PM is not None else "PM no especificado"
            }

    poly = a + b*T + c*T**2 + d*T**3 + e*T**4

    if c_ == "C_p":
        integrando = poly * R_gas
        k_kmol = "delta h [=] kJ/mol"
        k_kg = "delta h [=] kJ/kg"
        
    elif c_ == "C_v":
        integrando = poly * R_gas - R_gas
        k_kmol = "delta u [=] kJ/mol"
        k_kg = "delta u [=] kJ/kg"
        

    delta = (sp.integrate(integrando, (T, T_i, T_f)).evalf()) * 1000

    out = {
        k_kmol: delta
    }
    
    if PM is not None:
        out[k_kg] = delta / PM
    else:
        out[k_kg] = "PM no especificado"

    return out